In [12]:
import sys

sys.path.append("../../src/stong_core_cython")

In [13]:
import numpy as np
import pandas as pd
import restricted_hartree_fock
import basis_data
import importlib
import concurrent.futures
from scipy.optimize import minimize

importlib.reload(restricted_hartree_fock)
importlib.reload(basis_data)

from atom import *
from restricted_hartree_fock import *
from unrestricted_hartree_fock import *
from basis_data import *

In [14]:
df = pd.read_csv("../make_base/sto_3g.csv")

In [15]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 2, 0)}
    # hartreefock = RestrictedHartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
    hartreefock = UnrestrictedHartreeFock(
        molecule, basis_data, max_iter=10000, tol=1e-10
    )
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [16]:
get_energy([1.0, 0.1], "H")

Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])


-0.49494346138985884

In [17]:
elements = [
    "H",
    "He",
    "Li",
    "Be",
]
# elements = ["H"]
zetas = {}
results = {}
for i, element in enumerate(elements):
    print(element)
    if i == 0:
        result = minimize(get_energy, [5, 1], args=(element))
    else:
        result = minimize(get_energy, result.x, args=(element))
    # result = minimize(get_energy, [2.69, 0.75, 0.3], args=(element))
    results[element] = result
    zetas[element] = result.x
    print(zetas[element])

H
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, position=[0. 0. 0.])])
Start SCF for Molecule(atoms=[Atom(symbol=H, atomic_number=1, p

In [18]:
results

{'H':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -0.4966449228120141
         x: [ 1.348e+00  1.182e+00]
       nit: 16
       jac: [ 1.010e-06 -2.205e-06]
  hess_inv: [[ 5.338e+01  1.989e+01]
             [ 1.989e+01  1.471e+01]]
      nfev: 66
      njev: 22,
 'He':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -2.8399367658416383
         x: [ 2.411e+00  1.777e+00]
       nit: 11
       jac: [-5.931e-06  7.480e-06]
  hess_inv: [[ 1.745e+01  1.015e+01]
             [ 1.015e+01  8.432e+00]]
      nfev: 36
      njev: 12,
 'Li':   message: Optimization terminated successfully.
   success: True
    status: 0
       fun: -7.329116402898167
         x: [ 2.692e+00  6.523e-01]
       nit: 4
       jac: [ 5.603e-06  1.907e-06]
  hess_inv: [[ 5.535e-01 -8.710e-03]
             [-8.710e-03  8.979e-01]]
      nfev: 18
      njev: 6,
 'Be':   message: Optimization terminated successfully.
   success: 

In [19]:
df_zetas = pd.DataFrame(zetas)
df_zetas.to_csv("zetas.csv", index=False)